# Sentence to PClean query demo
This notebook demos the sentence-to-PClean pipeline. Given a sentence giving information about some doctor, it generates and runs queries about those people against the Medicare dataset used in Alex Lew's paper using a PClean model trained by Ian Limarta.


In [1]:
import gc
import json
import logging
from pathlib import Path
import sqlite3
import string
from typing import Any, Optional, TypeVar, Union

from IPython.display import display, Markdown
import lark
import nest_asyncio
import requests
import spacy
import torch
import transformers
from tqdm import tqdm
import vllm

import genparse
from genparse import InferenceSetup, InferenceSetupVLLM

nest_asyncio.apply()

logger = logging.getLogger("notebook")

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)

In [ ]:
debug_sentences_path = Path().resolve() / "debug_sentences.jsonl"
debug_sentences = [json.loads(line) for line in debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

In [ ]:
detailed_debug_sentences_path = Path().resolve() / "debug_sentences_detailed.jsonl"
detailed_debug_sentences = [json.loads(line) for line in detailed_debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

## Example inputs and outputs

Taken from the first ten sentences (rows 2-11) in the [GPT-4 tweets Marjorie Freedman generated][gpt4_tweets].

[gpt4_tweets]: https://docs.google.com/spreadsheets/d/1vq_HAdbFY079vYVOppkFLk_zojJ46MJKsy-TzUm7Q3M/edit

In [4]:
sentence_table_row_template = string.Template("| $sentence | <pre>$example_output</pre> |")
def show_sentences(sentence_data):
    lines = ["| Sentence | Example PClean preamble code |", "| --- | --- |"]
    for sentence_datum in sentence_data:
        lines.append(sentence_table_row_template.substitute(sentence=sentence_datum["sentence"], example_output=sentence_datum["example_output"].replace("\n", "<br>")))
    markdown = "\n".join(lines)
    display(Markdown(markdown))


show_sentences(debug_sentences)

| Sentence | Example PClean preamble code |
| --- | --- |
| Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."</pre> |
| Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "neurologist"</pre> |
| Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."</pre> |
| From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #MedicalCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "cardiologist"</pre> |
| Just wrapped up my appointment with Dr. Kay Ryan in Baltimore. Her professionalism is top-notch. #HealthJourney | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan's office in Baltimore is so efficient and welcoming. A great experience overall! #DoctorVisit #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Can't say enough good things about Dr. Kay Ryan, a pediatrician, in Baltimore. Truly an outstanding doctor! #Grateful | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "pediatrician"</pre> |

## Helper code

### SpaCy code

In [5]:
# Supporting code for entity extraction.
_SPACY_MODEL_NAME = 'en_core_web_trf'
spacy_model = spacy.load(_SPACY_MODEL_NAME)
_PERSON_LABEL = 'PERSON'
_LOCATION_LABEL = 'LOC'
# spaCy labels list example thanks to Stack Overflow user 'russhoppa': https://stackoverflow.com/a/78252807
_SPACY_LABELS = spacy_model.get_pipe("ner").labels
assert _PERSON_LABEL in _SPACY_LABELS

T = TypeVar('T')

def _uniquify(items: list[T]) -> list[T]:
    """
    O(n^2) order-preserving uniquification.

    Fine for short lists like a single sentence's list of PERSON entities.
    """
    result = []
    for item in items:
        if item not in result:
            result.append(item)
    return result

def get_people(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _PERSON_LABEL]
    )

def get_locations(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _LOCATION_LABEL]
    )

def show_ents(sentence: str) -> None:
    print(sentence)
    print(spacy_model(sentence).ents)

for sentence_datum in debug_sentences:
    show_ents(sentence_datum["sentence"])
    print()

Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore
(Kay Ryan, Baltimore, Baltimore)

Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare
(Kay Ryan, Baltimore, CityCare)

Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit
(Baltimore, Kay Ryan)

Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors
(Kay Ryan, today)

Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm
(Kay Ryan, Baltimore)

Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck
(Kay Ryan, Baltimore)

From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #Medical

### Extracting the generated code from a response

In [6]:
def extract_code_from_response(text: str) -> str:
    result = text.removeprefix('<|start_header_id|>assistant<|end_header_id|>').strip()
    return result

### Sort posterior

In [8]:
def sort_posterior(posterior):
    return {inference: likelihood for inference, likelihood in sorted(posterior.items(), key=lambda t: (t[1], t[0]), reverse=True)}

### Aggregate likelihoods over extracted code

In [66]:
def get_aggregate_likelihoods(posterior: dict[str, float]) -> dict[str, float]:
    result = {}
    for inference, likelihood in posterior.items():
        code_only = extract_code_from_response(inference)
        try:
            code_only = json.dumps(
                {key: value for key, value in sorted(json.loads(code_only).items(), key=lambda t: t[0])},
                indent=2
            )
        except json.JSONDecodeError:
            pass
        result.setdefault(code_only, 0.0)
        result[code_only] += likelihood
    return sort_posterior(result)

### Get best inference

In [10]:
def get_best_inference(posterior: dict[str, float]) -> tuple[str, float]:
    return max(posterior.items(), key=lambda t: (t[1], t[0]))

### Running inference locally

In [11]:
SERVER_MODEL_ID = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
SERVER_GENPARSE_MODEL_NAME = "llama3.1"
_LOCAL_BATCH_SIZE = 1

class FakeResponse:
    def __init__(self, data):
        self._data = data
    
    @property
    def status_code(self):
        return 200

    def json(self):
        return self._data

    def text(self):
        return json.dumps(self._data)

def server_model(grammar, proposal="character"):
    return InferenceSetupVLLM(SERVER_GENPARSE_MODEL_NAME, grammar, proposal_name=proposal)

def run_inference_vllm(
    prompt: str,
    *,
    proposal: str = 'character',
    batch_size: int = _LOCAL_BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str,
    genparse_url: str = "",
    inference_setup: InferenceSetupVLLM,
) -> requests.Response:
    #assert batch_size == inference_setup.sampler.llm._model.batch_size
    assert grammar == inference_setup.sampler.grammar
    assert (proposal == 'character' and isinstance(inference_setup, genparse.CharacterProposal))
    inference = inference_setup(prompt, n_particles=n_particles, temperature=temperature, max_tokens=max_tokens)
    return FakeResponse({"log_ml": None, "posterior": inference.posterior})

### Running the server

In [86]:
_BATCH_SIZE = 1
# Ben LeBrun's WIP server running on GCP as of 2024-07-16
_DEFAULT_GENPARSE_INFERENCE_SERVER_RESTART_URI = 'http://34.122.30.137:9999/'
_DEFAULT_GENPARSE_INFERENCE_SERVER_URI = 'http://34.122.30.137:8888/infer'
#_DEFAULT_GENPARSE_INFERENCE_SERVER_RESTART_URI = 'http://34.70.201.1:9999/'
#_DEFAULT_GENPARSE_INFERENCE_SERVER_URI = 'http://34.70.201.1:8888/infer'

def restart_inference_server():
    result = requests.get(_DEFAULT_GENPARSE_INFERENCE_SERVER_RESTART_URI)
    return result

def run_inference_server(
    prompt: str,
    *,
    proposal: str = 'character',
    batch_size: int = _BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str,
    genparse_url: str = _DEFAULT_GENPARSE_INFERENCE_SERVER_URI,
) -> requests.Response:
    """
    Run inference using a server.
    """
    params = {
        'prompt': prompt,
        'method': 'smc-standard',
        'n_particles': n_particles,
        'lark_grammar': grammar,
        'proposal_name': proposal,
        'proposal_args': {},
        'max_tokens': max_tokens,
        'temperature': temperature,
    }
    headers = {
        "Content-type": "application/json",
        "Accept": "application/json"
    }
    response = requests.post(genparse_url, json=params, headers=headers)

    return response

### JSON generation prompt

In [70]:
json_prompt = string.Template(
    """Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name)
- "last" (the doctor's surname)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et cetera -- "line 2" of the address)

Please generate a key-value pair for every such value specified in the sentence. Avoid nulls and empty strings. Omit any missing values. If the sentence calls the doctor "Dr. John Smith," include both the given and surname (`"first"` and `"last"`). If the sentence calls the doctor "Dr. Smith," include only the surname (`"last"`). Do not write a script. Output the JSON itself directly with no preamble or commentary. The following examples illustrate how you should behave on the input sentence.

Input: What a great experience with Dr. Williams's pediatrics office in Victoria! My child has never been happier to be jabbed with a needle. #ParentingVictories #Victoria
Output: ```json
{"last": "Williams", "specialty": "pediatrics", "city_name": "Victoria"}
```

Input: John Smith's neurology office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (Camp Hill) is terrible!
Output: ```json
{"first": "John", "last": "Smith", "specialty": "neurology", "legal_name": "Happy Brain Services LLC", "addr": "512 Example Street", "addr2": "Suite 3600", "city_name": "Camp Hill"}
```

Input: Loved visiting Dr. Kay Ryan's neurology office (Green Medicine Inc.) at 256 Overflow St (ZIP 17011-2202)! No wait time at all. #CampHill
Output: ```json
{"first": "Kay", "last": "Ryan", "legal_name": "Green Medicine Inc", "specialty": "neurology", "addr": "256 Overflow St", "zip": "170112202", "city_name": "Camp Hill"}
```

Input: Dr. Pat Rogers's office screwed us! So much for Soulful Medical Services Inc. Took our money and Pat gave us three minutes tops. #BaltimoreSucks
Output: ```json
{"first": "Pat", "last": "Rogers", "legal_name": "Soulful Medical Services Inc", "city_name": "Baltimore"}
```

Input: Dr. Maynard's really rules! They figured out my illness when no one else could. Loving the care here in #Lexington
Output: ```json
{"last": "Maynard", "city_name": "Lexington"}
```

The following is your input sentence. Produce the appropriate output.

Input: $sentence
Output:"""
)
# scrapped
# The doctor's name is $name. Their office address is $address. Their specialty is $specialty. The city is $city.

def format_json_prompt(sentence: str) -> str:
    return json_prompt.substitute(sentence=sentence)

print(json_prompt)

### JSON grammar

In [111]:
json_grammar = r"""
start: _form_no_fence
_form_no_fence: _ASSISTANT_TAG NL NL? NL? json
_ASSISTANT_TAG: "<|start_header_id|>assistant<|end_header_id|>"
json: "{" NL? OPTIONAL_WS kv_pair ("," NL? OPTIONAL_WS kv_pair) ~ 0..7 NL? OPTIONAL_WS "}"
kv_pair: COLUMN OPTIONAL_WS ":" OPTIONAL_WS value
value: STRING | NULL

CODE_FENCE: "```"
JSON_TAG: "json"
OPTIONAL_WS: WS? WS? WS? WS?
WS: " "
NL: "\n"
NULL: "null"
STRING: /"[a-zA-Z0-9-.,'& ]{0,60}"/
COLUMN: "\"" ("first" | "last" | "addr" | "addr2" | "specialty" | "city_name" | "zip" | "legal_name") "\""
"""

## Debugging

In [52]:
model_id = 'meta-llama/Meta-Llama-3.1-8B-Instruct'

In [53]:
def md_html_output_text_for_json(text: str) -> str:
    return text.replace("<", "&lt;").replace(">", "&gt;").replace("|", "\|").replace("\n", "<br>").replace("```", "\\```")

### Running the LM unconstrained

In [24]:
try:
    del model
    gc.collect()
except NameError:
    pass

In [27]:
# Taken from a magic number in the Genparse VLLM backend code.
_MAX_MODEL_LEN = 4096
# Why dtype=torch.float32? It's in the Genparse VLLM backend code.
model = vllm.LLM(model_id, dtype=torch.float32, max_model_len=_MAX_MODEL_LEN)

2024-07-30 13:45:05 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): huggingface.co:443
2024-07-30 13:45:05 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json HTTP/11" 200 0


INFO 07-30 13:45:05 config.py:1417] Upcasting torch.bfloat16 to torch.float32.
INFO 07-30 13:45:05 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float32, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=meta-llama/Meta-Llama-3.1-8B-Instruct, use_v2_block_manager=False, enable_prefix_caching=False)


2024-07-30 13:45:05 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer_config.json HTTP/11" 200 0
2024-07-30 13:45:05 - DEBUG - filelock -   Attempting to acquire lock 140039256926784 on /home/joseph/.cache/huggingface/hub/.locks/models--meta-llama--Meta-Llama-3.1-8B-Instruct/db88166e2bc4c799fd5d1ae643b75e84d03ee70e.lock
2024-07-30 13:45:05 - DEBUG - filelock -   Lock 140039256926784 acquired on /home/joseph/.cache/huggingface/hub/.locks/models--meta-llama--Meta-Llama-3.1-8B-Instruct/db88166e2bc4c799fd5d1ae643b75e84d03ee70e.lock
2024-07-30 13:45:05 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "GET /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer_config.json HTTP/11" 200 55351


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

2024-07-30 13:45:05 - DEBUG - filelock -   Attempting to release lock 140039256926784 on /home/joseph/.cache/huggingface/hub/.locks/models--meta-llama--Meta-Llama-3.1-8B-Instruct/db88166e2bc4c799fd5d1ae643b75e84d03ee70e.lock
2024-07-30 13:45:05 - DEBUG - filelock -   Lock 140039256926784 released on /home/joseph/.cache/huggingface/hub/.locks/models--meta-llama--Meta-Llama-3.1-8B-Instruct/db88166e2bc4c799fd5d1ae643b75e84d03ee70e.lock
2024-07-30 13:45:05 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer.json HTTP/11" 200 0
2024-07-30 13:45:05 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/added_tokens.json HTTP/11" 404 0
2024-07-30 13:45:05 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/special_tokens_map.json HTTP/11" 200 0
2024-07-30 13:45:06 - DEBUG - urlli

...

INFO 07-30 13:45:10 weight_utils.py:223] Using model weights format ['*.safetensors']


2024-07-30 13:45:11 - DEBUG - filelock -   Attempting to acquire lock 140039002714272 on /var/tmp/1e06ffef4faccfa02542c605236a96ba8ec984243111d7be1d4b1409c032448emeta-llama-Meta-Llama-3.1-8B-Instruct.lock
2024-07-30 13:45:11 - DEBUG - filelock -   Lock 140039002714272 acquired on /var/tmp/1e06ffef4faccfa02542c605236a96ba8ec984243111d7be1d4b1409c032448emeta-llama-Meta-Llama-3.1-8B-Instruct.lock
2024-07-30 13:45:11 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "GET /api/models/meta-llama/Meta-Llama-3.1-8B-Instruct/revision/main HTTP/11" 200 23946
2024-07-30 13:45:11 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): huggingface.co:443
2024-07-30 13:45:11 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): huggingface.co:443
2024-07-30 13:45:11 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): huggingface.co:443
2024-07-30 13:45:11 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): huggin

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-30 13:45:20 model_runner.py:692] Loading model weights took 29.9776 GB
INFO 07-30 13:45:25 gpu_executor.py:102] # GPU blocks: 725, # CPU blocks: 1024
INFO 07-30 13:45:27 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-30 13:45:27 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-30 13:45:51 model_runner.py:1181] Graph capturing finished in 24 secs.


2024-07-30 13:45:52 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): stats.vllm.ai:443
2024-07-30 13:45:52 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None
2024-07-30 13:55:52 - DEBUG - urllib3.connectionpool -   Resetting dropped connection: stats.vllm.ai
2024-07-30 13:55:52 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None


In [28]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

2024-07-30 13:45:51 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer_config.json HTTP/11" 200 0


In [44]:
sentence = debug_sentences[-1]["sentence"]
print(sentence)
people = get_people(sentence)
prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt(sentence=sentence)}], tokenize=False)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')

sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=128, n=25)
response = model.generate(prompt, sampling_params=sampling_params)[0]
for i, output in enumerate(response.outputs, start=1):
    print(f'Generated Query {i}: ```{output.text}```')

Loved visiting Dr. Kilmore's obstetrics practice in Camp Hill! Can't say enough good things.
Prompt (752 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name)
- "last" (the doctor's surname)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address infor

Processed prompts: 100%|███| 1/1 [00:02<00:00,  2.91s/it, est. speed input: 258.88 toks/s, output: 324.28 toks/s]

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Kilmore", 
  "specialty": "obstetrics", 
  "city_name": "Camp Hill"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Kilmore", 
  "specialty": "obstetrics", 
  "city_name": "Camp Hill"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Kilmore", 
  "specialty": "obstetrics", 
  "city_name": "Camp Hill"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Kilmore", 
  "specialty": "obstetrics", 
  "city_name": "Camp Hill"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Kilmore", 
  "specialty": "obstetrics", 
  "city_name": "Camp Hill"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Kilmore", 
  "specialty": "obstetrics", 
  "city_name": "Camp Hill"
}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>

{

In [98]:
sentence_to_outputs = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    print(sentence)
    prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt(sentence=sentence)}], tokenize=False)
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)
    
    sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=128, n=25)
    response = model.generate(prompt, sampling_params=sampling_params)[0]
    sentence_to_outputs[sentence] = []
    for i, output in enumerate(response.outputs, start=1):
        sentence_to_outputs[sentence].append(output)
        print(f'Generated Query {i}: ```{output.text}```')

2024-07-30 18:33:16 - DEBUG - notebook -   Prompt (780 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name)
- "last" (the doctor's surname)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et cete

Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore


Processed prompts: 100%|███| 1/1 [00:03<00:00,  3.34s/it, est. speed input: 233.27 toks/s, output: 253.91 toks/s]
2024-07-30 18:33:19 - DEBUG - notebook -   Prompt (780 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name)
- "last" (the doctor's surname)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" --

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{"first": "Kay", "last": "Ryan", "city_name": "Baltimore"}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{"first": "Kay", "last": "Ryan", "city_name": "Baltimore"}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{"first": "Kay", "last": "Ryan", "city_name": "Baltimore"}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{"first": "Kay", "last": "Ryan", "city_name": "Baltimore"}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{"first": "Kay", "last": "Ryan", "city_name": "Baltimore"}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{"first": "Kay", "last": "Ryan", "city_name": "Baltimore"}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>

{"first": "Kay", "last": "Ryan", "city_name": "Baltimore"}```
Generated Query 8: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "

Processed prompts: 100%|███| 1/1 [00:03<00:00,  3.38s/it, est. speed input: 230.82 toks/s, output: 278.46 toks/s]
2024-07-30 18:33:23 - DEBUG - notebook -   Prompt (782 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name)
- "last" (the doctor's surname)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" --

...

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{"last": "Jamison", "city_name": "Dallas", "specialty": "endocrinologist"}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{"last": "Jamison", "city_name": "Dallas", "specialty": "endocrinologist"}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{"last": "Jamison", "city_name": "Dallas", "specialty": "endocrinologist"}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{"last": "Jamison", "city_name": "Dallas", "specialty": "endocrinologist"}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{"last": "Jamison", "city_name": "Dallas", "specialty": "endocrinologist"}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{"last": "Jamison", "specialty": "endocrinologist", "city_name": "Dallas"}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>

{"last": "Jamison", "specialty": "endocrinologist

Processed prompts: 100%|███| 1/1 [00:02<00:00,  2.49s/it, est. speed input: 312.41 toks/s, output: 326.88 toks/s]
2024-07-30 18:34:14 - DEBUG - notebook -   Prompt (784 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name)
- "last" (the doctor's surname)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" --

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Thornton",
  "specialty": "ophthalmologist",
  "city_name": "LA"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Thornton",
  "specialty": "ophthalmologist",
  "city_name": "LA"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Thornton",
  "specialty": "ophthalmologist",
  "city_name": "LA"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Thornton",
  "specialty": "ophthalmologist",
  "city_name": "LA"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Thornton", 
  "specialty": "ophthalmologist", 
  "city_name": "LA"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Thornton", 
  "specialty": "ophthalmologist", 
  "city_name": "LA"
}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Th

Processed prompts: 100%|███| 1/1 [00:02<00:00,  2.68s/it, est. speed input: 292.77 toks/s, output: 334.59 toks/s]

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{"last": "Garrison", "specialty": "gastroenterologist", "city_name": "Phoenix"}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{"last": "Garrison", "specialty": "gastroenterologist", "city_name": "Phoenix"}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{"last": "Garrison", "specialty": "gastroenterologist", "city_name": "Phoenix"}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{"last": "Garrison", "specialty": "gastroenterologist", "city_name": "Phoenix"}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Garrison", 
  "specialty": "gastroenterologist", 
  "city_name": "Phoenix"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Garrison", 
  "specialty": "gastroenterologist", 
  "city_name": "Phoenix"
}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>


In [99]:
table_rows = ['| Sentence | Extracted Code | Raw Response | # Tokens | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:| ---:|']
for sentence, outputs in sentence_to_outputs.items():
    try:
        posterior = {}
        n_outputs = {}
        total_tokens = {}
        for i, output in enumerate(outputs):
            posterior.setdefault(output.text, 0.0)
            posterior[output.text] += 1.0 / len(outputs)
            n_outputs.setdefault(output.text, 0)
            n_outputs[output.text] += 1
            total_tokens.setdefault(output.text, 0)
            total_tokens[output.text] += len(output.token_ids)
        aggregate_likelihoods = sort_posterior(get_aggregate_likelihoods(posterior))
        n_outputs_aggregated = {}
        total_tokens_aggregated = {}
        for output in outputs:
            code = extract_code_from_response(output.text)
            try:
                code = json.dumps({key: value for key, value in sorted(json.loads(code).items())}, indent=2)
            except json.JSONDecodeError:
                pass
            n_outputs_aggregated.setdefault(code, 0)
            n_outputs_aggregated[code] += n_outputs[output.text]
            total_tokens_aggregated.setdefault(code, 0)
            total_tokens_aggregated[code] += total_tokens[output.text]
        for i, (code, likelihood) in enumerate(aggregate_likelihoods.items(), start=1):
            aggregate_likelihood = aggregate_likelihoods[code]
            code_with_br = code.replace("\n", "<br>")
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre>{code_with_br}</pre> '
                f'| {total_tokens_aggregated[code] / n_outputs_aggregated[code]}'
                f'| (unk) '
                f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError as e:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {e} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))

| Sentence | Extracted Code | Raw Response | # Tokens | Aggregate Likelihood (Raw LH) |
|:--- |:--- |:--- | ---:| ---:|
| Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 27.65934065934066| (unk) | 68.00% (68.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "specialty": "consultation"<br>}</pre> | 34.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": null<br>}</pre> | 40.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "unsupplied"<br>}</pre> | 40.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "unknown"<br>}</pre> | 41.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "consultation"<br>}</pre> | 39.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "null"<br>}</pre> | 41.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": null,<br>  "addr2": null,<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": null,<br>  "specialty": null,<br>  "zip": null<br>}</pre> | 63.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "null",<br>  "addr2": "null",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "null",<br>  "specialty": "null",<br>  "zip": "null"<br>}</pre> | 68.0| (unk) | 4.00% (4.00%) |
| Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare | <pre>{<br>  "addr": "321 Pine St.",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 37.94736842105263| (unk) | 44.00% (44.00%) |
|     | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 36.0| (unk) | 16.00% (16.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 30.4| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "addr": "321 Pine St.",<br>  "city_name": "Baltimore",<br>  "last": "Ryan"<br>}</pre> | 29.5| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "last": "Ryan"<br>}</pre> | 29.5| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "addr": "321 Pine St.",<br>  "addr2": null,<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": null<br>}</pre> | 58.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St.",<br>  "addr2": "",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": "office",<br>  "zip": ""<br>}</pre> | 64.0| (unk) | 4.00% (4.00%) |
|     | <pre>```json<br>{"first": "Kay", "last": "Ryan", "city_name": "Baltimore", "addr": "321 Pine St."}<br>```</pre> | 38.0| (unk) | 4.00% (4.00%) |
| Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 29.22| (unk) | 88.00% (88.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan"<br>}</pre> | 19.8| (unk) | 12.00% (12.00%) |
| Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "neurologist"<br>}</pre> | 40.146341463414636| (unk) | 80.00% (80.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": "neurologist"<br>}</pre> | 48.333333333333336| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "specialty": "neurologist"<br>}</pre> | 34.0| (unk) | 4.00% (4.00%) |
|     | <pre>```json<br>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurologist",<br>    "city_name": "Baltimore"<br>}<br>```</pre> | 45.0| (unk) | 4.00% (4.00%) |
| Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 28.74137931034483| (unk) | 80.00% (80.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": ""<br>}</pre> | 46.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "unknown"<br>}</pre> | 38.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": ""<br>}</pre> | 39.0| (unk) | 4.00% (4.00%) |
|     | <pre>```json<br>{"last": "Ryan", "city_name": "Baltimore"}<br>```</pre> | 23.0| (unk) | 4.00% (4.00%) |
| Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 36.867924528301884| (unk) | 68.00% (68.00%) |
|     | <pre>{<br>  "addr": "321 Pine St.",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 38.333333333333336| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 30.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "last": "Ryan"<br>}</pre> | 27.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": "",<br>  "zip": ""<br>}</pre> | 47.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": ""<br>}</pre> | 56.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St",<br>  "addr2": "Unknown",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "Unknown",<br>  "specialty": "Unknown",<br>  "zip": "Unknown"<br>}</pre> | 65.0| (unk) | 4.00% (4.00%) |
| From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #MedicalCare | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "cardiologist"<br>}</pre> | 40.607142857142854| (unk) | 72.00% (72.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": "cardiologist"<br>}</pre> | 46.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "addr": null,<br>  "addr2": null,<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": null,<br>  "specialty": "cardiologist",<br>  "zip": null<br>}</pre> | 73.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "specialty": "cardiologist"<br>}</pre> | 33.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "No legal name is mentioned in this sentence.",<br>  "specialty": "cardiologist"<br>}</pre> | 55.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 29.0| (unk) | 4.00% (4.00%) |
| Just wrapped up my appointment with Dr. Kay Ryan in Baltimore. Her professionalism is top-notch. #HealthJourney | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 30.01047120418848| (unk) | 84.00% (84.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan"<br>}</pre> | 19.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "addr": "undefined",<br>  "addr2": "undefined",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "undefined",<br>  "specialty": "undefined",<br>  "zip": "undefined"<br>}</pre> | 67.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "null",<br>  "addr2": "null",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "null",<br>  "specialty": "null"<br>}</pre> | 61.0| (unk) | 4.00% (4.00%) |
| Dr. Kay Ryan's office in Baltimore is so efficient and welcoming. A great experience overall! #DoctorVisit #CityCare | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 29.012820512820515| (unk) | 72.00% (72.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan"<br>}</pre> | 21.0| (unk) | 16.00% (16.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "specialty": "office"<br>}</pre> | 31.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "legal_name": null,<br>  "specialty": null<br>}</pre> | 31.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "legal_name": "CityCare"<br>}</pre> | 27.0| (unk) | 4.00% (4.00%) |
| Can't say enough good things about Dr. Kay Ryan, a pediatrician, in Baltimore. Truly an outstanding doctor! #Grateful | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "pediatrician"<br>}</pre> | 41.745098039215684| (unk) | 76.00% (76.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": "pediatrician"<br>}</pre> | 48.666666666666664| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "specialty": "pediatrician"<br>}</pre> | 33.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "No value available",<br>  "specialty": "pediatrician"<br>}</pre> | 43.0| (unk) | 4.00% (4.00%) |
| Loved visiting Dr. Kilmore's obstetrics practice in Camp Hill! Can't say enough good things. | <pre>{<br>  "city_name": "Camp Hill",<br>  "last": "Kilmore",<br>  "specialty": "obstetrics"<br>}</pre> | 31.908045977011493| (unk) | 88.00% (88.00%) |
|     | <pre>{<br>  "city_name": "Camp Hill",<br>  "first": "Kilmore",<br>  "last": "Kilmore",<br>  "specialty": "obstetrics"<br>}</pre> | 44.0| (unk) | 12.00% (12.00%) |
| Dr. Kilmore in Boston saved my life with his incredible cardiology skills. Forever grateful! #BostonDocs #HeartHealth | <pre>{<br>  "city_name": "Boston",<br>  "last": "Kilmore",<br>  "specialty": "cardiology"<br>}</pre> | 29.3855421686747| (unk) | 60.00% (60.00%) |
|     | <pre>{<br>  "city_name": "Boston",<br>  "first": "Kilmore",<br>  "last": "Kilmore",<br>  "specialty": "cardiology"<br>}</pre> | 41.63157894736842| (unk) | 36.00% (36.00%) |
|     | <pre>{<br>  "addr": null,<br>  "addr2": null,<br>  "city_name": "Boston",<br>  "first": "Kilmore",<br>  "last": "Kilmore",<br>  "legal_name": null,<br>  "specialty": "cardiology",<br>  "zip": null<br>}</pre> | 75.0| (unk) | 4.00% (4.00%) |
| Just had a life-changing neurology consult with Dr. Faraday in Seattle. Truly a genius! #SeattleScience #NeuroExperts | <pre>{<br>  "city_name": "Seattle",<br>  "last": "Faraday",<br>  "specialty": "neurology"<br>}</pre> | 29.206349206349206| (unk) | 52.00% (52.00%) |
|     | <pre>{<br>  "city_name": "Seattle",<br>  "first": "Faraday",<br>  "last": "Faraday",<br>  "specialty": "neurology"<br>}</pre> | 41.857142857142854| (unk) | 40.00% (40.00%) |
|     | <pre>{<br>  "first": "Faraday", "last": "Faraday", "specialty": "neurology", "city_name": "Seattle"</pre> | 37.0| (unk) | 4.00% (4.00%) |
|     | <pre>```<br>{"first": "Faraday", "last": "Faraday", "specialty": "neurology", "city_name": "Seattle"}<br>```</pre> | 38.0| (unk) | 4.00% (4.00%) |
| My skin has never been better thanks to Dr. Linwood in Denver. Highly recommend! #DenverDerm #SkinCare | <pre>{<br>  "city_name": "Denver",<br>  "first": "Linwood",<br>  "last": "Linwood"<br>}</pre> | 30.483870967741936| (unk) | 36.00% (36.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "last": "Linwood",<br>  "specialty": "dermatology"<br>}</pre> | 31.333333333333332| (unk) | 16.00% (16.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "last": "Linwood",<br>  "specialty": "derm"<br>}</pre> | 27.0| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "last": "Linwood",<br>  "specialty": "Derm"<br>}</pre> | 28.0| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "first": "Linwood",<br>  "last": "Linwood",<br>  "specialty": "derm"<br>}</pre> | 42.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "last": "Linwood",<br>  "specialty": "skin care"<br>}</pre> | 27.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "last": "Linwood"<br>}</pre> | 24.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "first": "Linwood",<br>  "last": "Linwood",<br>  "specialty": "dermatology"<br>}</pre> | 41.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "first": "Linwood",<br>  "last": "Linwood",<br>  "specialty": "Denver Dermatology Speciality"<br>}</pre> | 43.0| (unk) | 4.00% (4.00%) |
| Dr. Voss in Miami was amazing with my child’s treatment. Pediatric care at its finest! #MiamiMeds #KidsHealth | <pre>{<br>  "city_name": "Miami",<br>  "last": "Voss",<br>  "specialty": "pediatrics"<br>}</pre> | 30.0875| (unk) | 72.00% (72.00%) |
|     | <pre>{<br>  "city_name": "Miami",<br>  "first": "Voss",<br>  "last": "Voss",<br>  "specialty": "pediatrics"<br>}</pre> | 43.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "last": "Voss",<br>  "specialty": "pediatric care"<br>}</pre> | 26.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Miami",<br>  "first": "Voss",<br>  "last": "Voss",<br>  "specialty": "pediatric care"<br>}</pre> | 44.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Miami",<br>  "first": "Voss",<br>  "last": "",<br>  "specialty": "Pediatric care"<br>}</pre> | 40.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Miami",<br>  "first": "Dr. Voss",<br>  "specialty": "pediatrics"<br>}</pre> | 36.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": null,<br>  "addr2": null,<br>  "city_name": "Miami",<br>  "first": "Voss",<br>  "last": "Voss",<br>  "legal_name": null,<br>  "specialty": "pediatrics",<br>  "zip": null<br>}</pre> | 66.0| (unk) | 4.00% (4.00%) |
| Dr. Ravenswood in San Francisco fixed my knee like a pro. Orthopedic wizard! #SFOOrtho #BoneHealth | <pre>{<br>  "city_name": "San Francisco",<br>  "last": "Ravenswood",<br>  "specialty": "Orthopedic"<br>}</pre> | 34.63636363636363| (unk) | 40.00% (40.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "last": "Ravenswood",<br>  "specialty": "orthopedic"<br>}</pre> | 35.4| (unk) | 32.00% (32.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "last": "Ravenswood",<br>  "specialty": "orthopedics"<br>}</pre> | 31.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "last": "Ravenswood",<br>  "specialty": "Orthopedics"<br>}</pre> | 38.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "first": "Ravenswood",<br>  "specialty": "orthopedic"<br>}</pre> | 38.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "first": "Ravenswood",<br>  "last": null,<br>  "specialty": "Orthopedic"<br>}</pre> | 45.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "first": "Dr. Ravenswood",<br>  "last": "",<br>  "specialty": "Orthopedic"<br>}</pre> | 38.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "first": "",<br>  "last": "Ravenswood",<br>  "specialty": "Orthopedic"<br>}</pre> | 41.0| (unk) | 4.00% (4.00%) |
|     | <pre>```json<br>{"first": "Ravenswood", "last": "", "specialty": "Orthopedic", "city_name": "San Francisco"}<br>```</pre> | 39.0| (unk) | 4.00% (4.00%) |
| Grateful for Dr. Shapiro in Chicago who guided me through my cancer journey. The best oncologist! #ChicagoCare #FightCancer | <pre>{<br>  "city_name": "Chicago",<br>  "last": "Shapiro",<br>  "specialty": "oncologist"<br>}</pre> | 32.05882352941177| (unk) | 48.00% (48.00%) |
|     | <pre>{<br>  "city_name": "Chicago",<br>  "first": "Dr. Shapiro",<br>  "last": "Shapiro",<br>  "specialty": "oncologist"<br>}</pre> | 43.92857142857143| (unk) | 24.00% (24.00%) |
|     | <pre>{<br>  "city_name": "Chicago",<br>  "first": "Shapiro",<br>  "last": "Shapiro",<br>  "specialty": "oncologist"<br>}</pre> | 38.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Chicago",<br>  "last": "Shapiro",<br>  "specialty": "oncology"<br>}</pre> | 35.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Chicago",<br>  "first": "Shapiro",<br>  "specialty": "oncologist"<br>}</pre> | 35.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Chicago",<br>  "first": "Dr. Shapiro",<br>  "last": "Shapiro",<br>  "legal_name": "Grateful for Dr. Shapiro",<br>  "specialty": "oncologist"<br>}</pre> | 55.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Chicago",<br>  "first": "Dr. Shapiro",<br>  "last": "Shapiro"<br>}</pre> | 34.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "",<br>  "addr2": "",<br>  "city_name": "Chicago",<br>  "first": "Shapiro",<br>  "last": "",<br>  "legal_name": "",<br>  "specialty": "oncologist",<br>  "zip": ""<br>}</pre> | 67.0| (unk) | 4.00% (4.00%) |
| Feeling mentally strong thanks to Dr. Everhart in NYC. Exceptional psychiatrist! #NYCWellness #MentalHealth | <pre>{<br>  "city_name": "NYC",<br>  "last": "Everhart",<br>  "specialty": "psychiatrist"<br>}</pre> | 31.58823529411765| (unk) | 68.00% (68.00%) |
|     | <pre>{<br>  "city_name": "NYC",<br>  "first": "Everhart",<br>  "last": "Everhart",<br>  "specialty": "psychiatrist"<br>}</pre> | 42.0| (unk) | 16.00% (16.00%) |
|     | <pre>{<br>  "first": "Dr. Everhart",<br>  "last": "Everhart",<br>  "specialty": "psychiatrist", "city_name": "NYC"</pre> | 42.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "NYC",<br>  "first": "Everhart",<br>  "specialty": "psychiatrist"<br>}</pre> | 29.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "NYC",<br>  "first": "Dr. Everhart",<br>  "last": "Everhart",<br>  "specialty": "psychiatrist"<br>}</pre> | 44.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": null,<br>  "addr2": null,<br>  "city_name": "NYC",<br>  "first": "Everhart",<br>  "last": "Everhart",<br>  "legal_name": null,<br>  "specialty": "psychiatrist",<br>  "zip": null<br>}</pre> | 75.0| (unk) | 4.00% (4.00%) |
| Dr. Jamison in Dallas balanced my hormones perfectly. Best endocrinologist ever! #DallasDocs #HormoneHealth | <pre>{<br>  "city_name": "Dallas",<br>  "last": "Jamison",<br>  "specialty": "endocrinologist"<br>}</pre> | 28.338461538461537| (unk) | 60.00% (60.00%) |
|     | <pre>{<br>  "city_name": "Dallas",<br>  "first": "Jamison",<br>  "last": "Jamison",<br>  "specialty": "endocrinologist"<br>}</pre> | 43.42857142857143| (unk) | 20.00% (20.00%) |
|     | <pre>{<br>  "city_name": "Dallas",<br>  "first": "Jamison",<br>  "last": "Jamison"<br>}</pre> | 33.0| (unk) | 16.00% (16.00%) |
|     | <pre>{<br>  "city_name": "Dallas",<br>  "last": "Jamison"<br>}</pre> | 19.0| (unk) | 4.00% (4.00%) |
| Seeing clearly again thanks to Dr. Thornton in LA. Incredible ophthalmologist! #LAVisions #EyeCare | <pre>{<br>  "city_name": "LA",<br>  "last": "Thornton",<br>  "specialty": "ophthalmologist"<br>}</pre> | 31.26923076923077| (unk) | 64.00% (64.00%) |
|     | <pre>{<br>  "city_name": "LA",<br>  "first": "Thornton",<br>  "last": "Thornton",<br>  "specialty": "ophthalmologist"<br>}</pre> | 39.8| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "city_name": "LA",<br>  "last": "Thornton"<br>}</pre> | 21.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "LA",<br>  "last": "Thornton",<br>  "specialty": "ophthalmology"<br>}</pre> | 35.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "LA",<br>  "first": "Thornton",<br>  "last": "Thornton"<br>}</pre> | 33.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "LA",<br>  "first": "Dr. Thornton",<br>  "last": "Thornton",<br>  "specialty": "ophthalmologist"<br>}</pre> | 42.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "LA",<br>  "first": "Dr. Thornton",<br>  "last": " Thornton",<br>  "specialty": "ophthalmologist"<br>}</pre> | 41.0| (unk) | 4.00% (4.00%) |
| Dr. Garrison in Phoenix fixed my indigestion and it only took one visit. Best gastroenterologist around! #PhoenixHealth #DigestiveCare | <pre>{<br>  "city_name": "Phoenix",<br>  "last": "Garrison",<br>  "specialty": "gastroenterologist"<br>}</pre> | 34.28| (unk) | 72.00% (72.00%) |
|     | <pre>{<br>  "city_name": "Phoenix",<br>  "first": "Garrison",<br>  "last": "Garrison",<br>  "specialty": "gastroenterologist"<br>}</pre> | 43.857142857142854| (unk) | 20.00% (20.00%) |
|     | <pre>{<br>  "city_name": "Phoenix",<br>  "last": "Garrison",<br>  "specialty": "gastroenterology"<br>}</pre> | 36.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Phoenix",<br>  "first": "Garrison",<br>  "last": "Garrison",<br>  "specialty": "gastroenterology"<br>}</pre> | 45.0| (unk) | 4.00% (4.00%) |

In [ ]:
try:
    del model
    gc.collect()
except NameError:
    pass

### Running the model with constraints

In [ ]:
try:
    tokenizer
except NameError:
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

In [46]:
sentence = debug_sentences[-1]["sentence"]
prompt = tokenizer.apply_chat_template(
    [
        {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
    ],
    tokenize=False,
)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')


# ignored top_p=0.95
server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
response = run_inference_server(prompt, **server_inference_params)
try:
    data = response.json()
    for i, (query, likelihood) in enumerate(data['posterior'].items(), start=1):
        
        print(f'Generated Query {i} (likelihood {100 * likelihood:.2f}%): ```{query}```')
except json.JSONDecodeError:
    print(response.status_code)
    print(response.text)

2024-07-30 13:57:44 - DEBUG - urllib3.connectionpool -   Starting new HTTP connection (1): 34.122.30.137:8888


Prompt (752 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name)
- "last" (the doctor's surname)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et cetera -- "line 2" of the address)

Please gene

2024-07-30 13:57:50 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:8888 "POST /infer HTTP/11" 200 1585


Generated Query 1 (likelihood 16.46%): ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kilmore",
    "last": "Kilmore",
    "specialty": "obstetrics",
    "city_name": "Camp Hill"
}```
Generated Query 2 (likelihood 0.17%): ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kilmore",
    "last": "Kilmore",
    "specialty": "obstetrics",
    "city_name": "Camp Hill",
    "addr": "Curabitur id sem vel dui", "addr2": "Suada a iaculis mauris."
}```
Generated Query 3 (likelihood 1.06%): ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kilmore",
    "last": "Kilmore",
    "specialty": "obstetrics",
    "city_name": "Camp Hill",
    "addr": "Curious Cape Street"
}```
Generated Query 4 (likelihood 8.23%): ```<|start_header_id|>assistant<|end_header_id|>

{
    "last": "Kilmore",
    "specialty": "obstetrician",
    "city_name": "Camp Hill"
}```
Generated Query 5 (likelihood 8.23%): ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "

In [107]:
responses = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response
    print(response)

2024-07-30 19:09:40 - DEBUG - notebook -   Prompt (780 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name)
- "last" (the doctor's surname)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et cete

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [ ]:
table_rows = ['| Sentence | Extracted Code | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        aggregate_likelihoods = get_aggregate_likelihoods(posterior)
        # for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
        #     # Assert the response parses properly
        #     pclean_parser.parse(output_text)

        #     code = extract_code_from_response(output_text)
        #     aggregate_likelihood = aggregate_likelihoods[code]
        #     code_with_br = code.replace("\n", "<br>")
        #     output_with_br = md_html_output_text(output_text)
        #     table_rows.append(
        #         f'| {sentence if i == 1 else "   "} '
        #         f'| <pre>{code_with_br}</pre> '
        #         f'| <pre>{output_with_br}</pre> '
        #         f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
        #     )
        for i, (code, likelihood) in enumerate(aggregate_likelihoods.items(), start=1):
            aggregate_likelihood = aggregate_likelihoods[code]
            code_with_br = code.replace("\n", "<br>")
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre>{code_with_br}</pre> '
                f'| (unk) '
                f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {response.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))

### Local inference

In [ ]:
inference_setup = server_model(json_grammar)

In [ ]:
print(inference_setup.sampler.llm._model.batch_size)

In [ ]:
dir(inference_setup.sampler.llm._model)

In [ ]:
responses = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_vllm(prompt, inference_setup=inference_setup, **server_inference_params)
    responses[sentence] = response

In [ ]:
try:
    del inference_setup
except NameError:
    pass

### Running the model on an arbitrary sentence

In [ ]:
responses = {}
for sentence_datum in tqdm([#{"sentence": "John Smith the podiatrist is terrible, so much worse than John Smith the cardiologist."}, {"sentence": "John Smith works with Jack Smith at the neurology office. He’s terrible at neurology."},
                            {"sentence": "John Smith's office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (CA-170) is terrible!"}]):
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 256, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response
    #break

In [ ]:
table_rows = ['| Sentence | Extracted JSON | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
            # Assert the response parses properly
            # TODO grammar no longer parses in lark but does in genparse so idk, can't test this as easily
            #json_parser.parse(output_text)

            #code = extract_code_from_response(output_text)
            code = output_text
            code_with_br = code.replace("\n", "<br>")
            output_with_br = md_html_output_text_for_json(output_text)
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre></pre> '
                f'| <pre>{output_with_br}</pre> '
                f'| {100 * likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {response.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))